In [18]:
pip install telepot
pip install telepot --upgrade  # 更新
pip install pprint # 讓 print 出來的文字自動排版

     |████████████████████████████████| 73 kB 272 kB/s eta 0:00:01
     |████████████████████████████████| 642 kB 567 kB/s eta 0:00:01
     |████████████████████████████████| 125 kB 760 kB/s eta 0:00:01
     |████████████████████████████████| 48 kB 1.2 MB/s eta 0:00:011
  Created wheel for telepot: filename=telepot-12.7-py3-none-any.whl size=57962 sha256=0839f0e33ef4fcbf560369cbc878e9b488401a669ac7eeb47fc7b58907dc98be
  Stored in directory: /Users/Rebeca/Library/Caches/pip/wheels/59/4d/ea/8781198a69408b6c37d47d902d7bbe5541969eddf5290e6b2e
Successfully built telepot
Note: you may need to restart the kernel to use updated packages.


In [23]:
import telepot
bot = telepot.Bot('1151794661:AAGbQ1sM7_XwhKfz8sw6aWBjVyX5q_DDdRY')
print(bot.getMe())

{'id': 1151794661, 'is_bot': True, 'first_name': 'Fintech', 'username': 'Fintech_scu_bot', 'can_join_groups': True, 'can_read_all_group_messages': False, 'supports_inline_queries': False}


In [35]:
import requests
import configparser
import telegram
from flask import Flask, request
from telegram.ext import Dispatcher, MessageHandler, Filters, Updater, CommandHandler, InlineQueryHandler, CommandHandler
from fugle_realtime import intraday
from datetime import datetime,timedelta
import pandas as pd
import sys
import matplotlib.pyplot as plt
import time
import networkx as nx
import matplotlib.pyplot as plt

In [36]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [37]:
access_token = config['TELEGRAM']['ACCESS_TOKEN']
webhook_url = config['TELEGRAM']['WEBHOOK_URL']

In [38]:
requests.post('https://api.telegram.org/bot'+access_token+'/deleteWebhook').text

'{"ok":true,"result":true,"description":"Webhook was deleted"}'

In [39]:
requests.post('https://api.telegram.org/bot'+access_token+'/setWebhook?url='+webhook_url+'/hook').text

'{"ok":true,"result":true,"description":"Webhook was set"}'

In [40]:
def grapg(text):
        stock_sim=pd.read_csv('stock_similarity/'+text+'.csv',index_col=0)

        dic = {'From':[],'to':[],'relation':[]}

        for n in range(len(stock_sim)):
            dic['From'].append(str(stock_sim['From'][n]))
            dic['to'].append(str(stock_sim['to'][n]))
            dic['relation'].append(stock_sim['relation'][n])

        for i in stock_sim['to']:
            K_data = intraday.chart(apiToken=api,symbolId=i)
            close=K_data.iloc[0]['close']
            high=K_data.iloc[0]['high']
            low=K_data.iloc[0]['low']
            open=K_data.iloc[0]['open']
            unit=K_data.iloc[0]['unit']

            for n in range(5):
                dic['From'].append(str(i))
            dic['to'].append(str(close))
            dic['to'].append(str(high))
            dic['to'].append(str(low))
            dic['to'].append(str(open))
            dic['to'].append(str(unit))
            dic['relation'].append('close')
            dic['relation'].append('high')
            dic['relation'].append('low')
            dic['relation'].append('open')
            dic['relation'].append('unit')

        df = pd.DataFrame(dic)

        #繪圖
        node_dic = {}
        for i in df['From']:
            node_dic[str(i)]=[]
            for j in df[df['From']==i]['to']:
                node_dic[i].append(j)

        layout={}

        #中心連出去
        layout[text]=[0,0]
        layout[node_dic[text][0]]=[0,0.175] #1234
        layout[node_dic[text][1]]=[0.0875,0.0875]
        layout[node_dic[text][2]]=[0.0875,-0.0875]
        layout[node_dic[text][3]]=[-0.0875,0.0875]
        layout[node_dic[text][4]]=[-0.0875,-0.0875]


        #第一支相關聯的股票
        layout[node_dic[node_dic[text][0]][0]]=[-0.04,0.25]
        layout[node_dic[node_dic[text][0]][1]]=[-0.025,0.275]
        layout[node_dic[node_dic[text][0]][2]]=[0,0.29]
        layout[node_dic[node_dic[text][0]][3]]=[0.025,0.275]
        layout[node_dic[node_dic[text][0]][4]]=[0.04,0.25]

        layout[node_dic[node_dic[text][1]][0]]=[0.125,0.165]
        layout[node_dic[node_dic[text][1]][1]]=[0.14,0.175]
        layout[node_dic[node_dic[text][1]][2]]=[0.165,0.165]
        layout[node_dic[node_dic[text][1]][3]]=[0.175,0.14]
        layout[node_dic[node_dic[text][1]][4]]=[0.165,0.125]

        layout[node_dic[node_dic[text][2]][0]]=[0.165,-0.125]
        layout[node_dic[node_dic[text][2]][1]]=[0.175,-0.14]
        layout[node_dic[node_dic[text][2]][2]]=[0.165,-0.165]
        layout[node_dic[node_dic[text][2]][3]]=[0.14,-0.175]
        layout[node_dic[node_dic[text][2]][4]]=[0.125,-0.165]

        layout[node_dic[node_dic[text][3]][0]]=[-0.125,0.165]
        layout[node_dic[node_dic[text][3]][1]]=[-0.14,0.175]
        layout[node_dic[node_dic[text][3]][2]]=[-0.165,0.165]
        layout[node_dic[node_dic[text][3]][3]]=[-0.175,0.14]
        layout[node_dic[node_dic[text][3]][4]]=[-0.165,0.125]


        layout[node_dic[node_dic[text][4]][0]]=[-0.165,-0.125]
        layout[node_dic[node_dic[text][4]][1]]=[-0.175,-0.14]
        layout[node_dic[node_dic[text][4]][1]]=[-0.175,-0.14]
        layout[node_dic[node_dic[text][4]][2]]=[-0.165,-0.165]
        layout[node_dic[node_dic[text][4]][3]]=[-0.14,-0.175]
        layout[node_dic[node_dic[text][4]][4]]=[-0.125,-0.165]
        
        return(df)

def draw_kg(pairs):
    k_graph = nx.from_pandas_edgelist(pairs, 'From', 'to',
            create_using=nx.MultiDiGraph())

    node_deg = nx.degree(k_graph)
    #layout = nx.spring_layout(k_graph) #nx.spring_layout(k_graph)#, k=0.15, iterations=20)
    plt.figure(num=None, figsize=(5, 5))#, dpi=80)

    nx.draw_networkx(
        k_graph,
        node_size=300,#[int(deg[1]) * 500 for deg in node_deg],
        arrowsize=2,
        linewidths=1,
        pos=layout,
        edge_color='red',
        edgecolors='black',
        node_color='white',
        font_size=5
        )
    labels = dict(zip(list(zip(pairs.From, pairs.to)),
                  pairs['relation'].tolist()))
    nx.draw_networkx_edge_labels(k_graph, pos=layout, edge_labels=labels,font_size=5,
                                 font_color='g')
    plt.axis('off')
    #plt.show()
    plt.savefig('613.png')


In [ ]:
# Initial Flask app
app = Flask(__name__)

# Initial bot by Telegram access token
bot = telegram.Bot(token=config['TELEGRAM']['ACCESS_TOKEN'])
api = '51714d20eae11ba0c9c9647cab45da4b'

@app.route('/hook', methods=['POST'])
def webhook_handler():
    if request.method == "POST":
        update = telegram.Update.de_json(request.get_json(force=True), bot)
        # Update dispatcher process that handler to process this message
        dispatcher.process_update(update)
    return 'ok'

## reply message
def reply_handler(bot, update):
    text = update.message.text #使用者鍵入的值
    if text == '/start':
        #update.message.reply_text('do u want /search stock symbolId or check stock /data?')
        d = pd.read_csv('成交金額前20大.csv',index_col=0)
        update.message.reply_text('請選擇下列其中一支股票：\n'+d.to_string(index=False))
    else:
        d = pd.read_csv('成交金額前20大.csv',index_col=0)
        d_l = list(d['symbol'])
        if text in d_l:
            text=text.split("/")[1]
#             df = graph(text)
#             draw_kg(df)
#             update.message.reply_text('/graph')
#         elif text == '/graph':
            chat_id = update.message.chat_id
            bot.send_photo(chat_id=chat_id, photo=open(str(text)+'.png', 'rb'))

        else:
            update.message.reply_text('I don\'t know what u want')
            
            #update.message.reply_text('/graph')
            
                
#             stock_sim=pd.read_csv('stock_similarity/'+text+'.csv',index_col=0)

#             dic = {'From':[],'to':[],'relation':[]}

#             for n in range(len(stock_sim)):
#                 dic['From'].append(str(stock_sim['From'][n]))
#                 dic['to'].append(str(stock_sim['to'][n]))
#                 dic['relation'].append(stock_sim['relation'][n])

#             for i in stock_sim['to']:
#                 K_data = intraday.chart(apiToken=api,symbolId=i)
#                 close=K_data.iloc[0]['close']
#                 high=K_data.iloc[0]['high']
#                 low=K_data.iloc[0]['low']
#                 open=K_data.iloc[0]['open']
#                 unit=K_data.iloc[0]['unit']

#                 for n in range(5):
#                     dic['From'].append(str(i))
#                 dic['to'].append(str(close))
#                 dic['to'].append(str(high))
#                 dic['to'].append(str(low))
#                 dic['to'].append(str(open))
#                 dic['to'].append(str(unit))
#                 dic['relation'].append('close')
#                 dic['relation'].append('high')
#                 dic['relation'].append('low')
#                 dic['relation'].append('open')
#                 dic['relation'].append('unit')

#             df = pd.DataFrame(dic)

#             #繪圖
#             node_dic = {}
#             for i in df['From']:
#                 node_dic[str(i)]=[]
#                 for j in df[df['From']==i]['to']:
#                     node_dic[i].append(j)

#             keyword = text
#             layout={}

#             #中心連出去
#             layout[keyword]=[0,0]
#             layout[node_dic[keyword][0]]=[0,0.175] #1234
#             layout[node_dic[keyword][1]]=[0.0875,0.0875]
#             layout[node_dic[keyword][2]]=[0.0875,-0.0875]
#             layout[node_dic[keyword][3]]=[-0.0875,0.0875]
#             layout[node_dic[keyword][4]]=[-0.0875,-0.0875]


#             #第一支相關聯的股票
#             layout[node_dic[node_dic[keyword][0]][0]]=[-0.04,0.25]
#             layout[node_dic[node_dic[keyword][0]][1]]=[-0.025,0.275]
#             layout[node_dic[node_dic[keyword][0]][2]]=[0,0.29]
#             layout[node_dic[node_dic[keyword][0]][3]]=[0.025,0.275]
#             layout[node_dic[node_dic[keyword][0]][4]]=[0.04,0.25]

#             layout[node_dic[node_dic[keyword][1]][0]]=[0.125,0.165]
#             layout[node_dic[node_dic[keyword][1]][1]]=[0.14,0.175]
#             layout[node_dic[node_dic[keyword][1]][2]]=[0.165,0.165]
#             layout[node_dic[node_dic[keyword][1]][3]]=[0.175,0.14]
#             layout[node_dic[node_dic[keyword][1]][4]]=[0.165,0.125]

#             layout[node_dic[node_dic[keyword][2]][0]]=[0.165,-0.125]
#             layout[node_dic[node_dic[keyword][2]][1]]=[0.175,-0.14]
#             layout[node_dic[node_dic[keyword][2]][2]]=[0.165,-0.165]
#             layout[node_dic[node_dic[keyword][2]][3]]=[0.14,-0.175]
#             layout[node_dic[node_dic[keyword][2]][4]]=[0.125,-0.165]

#             layout[node_dic[node_dic[keyword][3]][0]]=[-0.125,0.165]
#             layout[node_dic[node_dic[keyword][3]][1]]=[-0.14,0.175]
#             layout[node_dic[node_dic[keyword][3]][2]]=[-0.165,0.165]
#             layout[node_dic[node_dic[keyword][3]][3]]=[-0.175,0.14]
#             layout[node_dic[node_dic[keyword][3]][4]]=[-0.165,0.125]


#             layout[node_dic[node_dic[keyword][4]][0]]=[-0.165,-0.125]
#             layout[node_dic[node_dic[keyword][4]][1]]=[-0.175,-0.14]
#             layout[node_dic[node_dic[keyword][4]][1]]=[-0.175,-0.14]
#             layout[node_dic[node_dic[keyword][4]][2]]=[-0.165,-0.165]
#             layout[node_dic[node_dic[keyword][4]][3]]=[-0.14,-0.175]
#             layout[node_dic[node_dic[keyword][4]][4]]=[-0.125,-0.165]

#             def draw_kg(pairs):
#                 k_graph = nx.from_pandas_edgelist(pairs, 'From', 'to',
#                         create_using=nx.MultiDiGraph())

#                 node_deg = nx.degree(k_graph)
#                 #layout = nx.spring_layout(k_graph) #nx.spring_layout(k_graph)#, k=0.15, iterations=20)
#                 plt.figure(num=None, figsize=(5, 5))#, dpi=80)

#                 nx.draw_networkx(
#                     k_graph,
#                     node_size=300,#[int(deg[1]) * 500 for deg in node_deg],
#                     arrowsize=2,
#                     linewidths=1,
#                     pos=layout,
#                     edge_color='red',
#                     edgecolors='black',
#                     node_color='white',
#                     font_size=5
#                     )
#                 labels = dict(zip(list(zip(pairs.From, pairs.to)),
#                               pairs['relation'].tolist()))
#                 nx.draw_networkx_edge_labels(k_graph, pos=layout, edge_labels=labels,font_size=5,
#                                              font_color='g')
#                 plt.axis('off')
#                 #plt.show()
#                 plt.savefig('gr.png')
            
            
# This class dispatches all kinds of updates to its registered handlers.
dispatcher = Dispatcher(bot, None)
dispatcher.add_handler(MessageHandler(Filters.text, reply_handler))

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2020 19:32:24] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 19:33:56] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 19:34:06] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 19:34:12] "POST /hook HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 19:39:26] "POST /hook HTTP/1.1" 200 -


In [57]:
text = '2330'
def grapg(text):
        stock_sim=pd.read_csv('stock_similarity/'+text+'.csv',index_col=0)

        dic = {'From':[],'to':[],'relation':[]}

        for n in range(len(stock_sim)):
            dic['From'].append(str(stock_sim['From'][n]))
            dic['to'].append(str(stock_sim['to'][n]))
            dic['relation'].append(stock_sim['relation'][n])

        for i in stock_sim['to']:
            K_data = intraday.chart(apiToken=api,symbolId=i)
            close=K_data.iloc[0]['close']
            high=K_data.iloc[0]['high']
            low=K_data.iloc[0]['low']
            open=K_data.iloc[0]['open']
            unit=K_data.iloc[0]['unit']

            for n in range(5):
                dic['From'].append(str(i))
            dic['to'].append(str(close))
            dic['to'].append(str(high))
            dic['to'].append(str(low))
            dic['to'].append(str(open))
            dic['to'].append(str(unit))
            dic['relation'].append('close')
            dic['relation'].append('high')
            dic['relation'].append('low')
            dic['relation'].append('open')
            dic['relation'].append('unit')

        df = pd.DataFrame(dic)

        #繪圖
        node_dic = {}
        for i in df['From']:
            node_dic[str(i)]=[]
            for j in df[df['From']==i]['to']:
                node_dic[i].append(j)

        layout={}

        #中心連出去
        layout[text]=[0,0]
        layout[node_dic[text][0]]=[0,0.175] #1234
        layout[node_dic[text][1]]=[0.0875,0.0875]
        layout[node_dic[text][2]]=[0.0875,-0.0875]
        layout[node_dic[text][3]]=[-0.0875,0.0875]
        layout[node_dic[text][4]]=[-0.0875,-0.0875]


        #第一支相關聯的股票
        layout[node_dic[node_dic[text][0]][0]]=[-0.04,0.25]
        layout[node_dic[node_dic[text][0]][1]]=[-0.025,0.275]
        layout[node_dic[node_dic[text][0]][2]]=[0,0.29]
        layout[node_dic[node_dic[text][0]][3]]=[0.025,0.275]
        layout[node_dic[node_dic[text][0]][4]]=[0.04,0.25]

        layout[node_dic[node_dic[text][1]][0]]=[0.125,0.165]
        layout[node_dic[node_dic[text][1]][1]]=[0.14,0.175]
        layout[node_dic[node_dic[text][1]][2]]=[0.165,0.165]
        layout[node_dic[node_dic[text][1]][3]]=[0.175,0.14]
        layout[node_dic[node_dic[text][1]][4]]=[0.165,0.125]

        layout[node_dic[node_dic[text][2]][0]]=[0.165,-0.125]
        layout[node_dic[node_dic[text][2]][1]]=[0.175,-0.14]
        layout[node_dic[node_dic[text][2]][2]]=[0.165,-0.165]
        layout[node_dic[node_dic[text][2]][3]]=[0.14,-0.175]
        layout[node_dic[node_dic[text][2]][4]]=[0.125,-0.165]

        layout[node_dic[node_dic[text][3]][0]]=[-0.125,0.165]
        layout[node_dic[node_dic[text][3]][1]]=[-0.14,0.175]
        layout[node_dic[node_dic[text][3]][2]]=[-0.165,0.165]
        layout[node_dic[node_dic[text][3]][3]]=[-0.175,0.14]
        layout[node_dic[node_dic[text][3]][4]]=[-0.165,0.125]


        layout[node_dic[node_dic[text][4]][0]]=[-0.165,-0.125]
        layout[node_dic[node_dic[text][4]][1]]=[-0.175,-0.14]
        layout[node_dic[node_dic[text][4]][1]]=[-0.175,-0.14]
        layout[node_dic[node_dic[text][4]][2]]=[-0.165,-0.165]
        layout[node_dic[node_dic[text][4]][3]]=[-0.14,-0.175]
        layout[node_dic[node_dic[text][4]][4]]=[-0.125,-0.165]
        
        return(df)

def draw_kg(pairs):
    k_graph = nx.from_pandas_edgelist(pairs, 'From', 'to',
            create_using=nx.MultiDiGraph())

    node_deg = nx.degree(k_graph)
    #layout = nx.spring_layout(k_graph) #nx.spring_layout(k_graph)#, k=0.15, iterations=20)
    plt.figure(num=None, figsize=(5, 5))#, dpi=80)

    nx.draw_networkx(
        k_graph,
        node_size=300,#[int(deg[1]) * 500 for deg in node_deg],
        arrowsize=2,
        linewidths=1,
        pos=layout,
        edge_color='red',
        edgecolors='black',
        node_color='white',
        font_size=5
        )
    labels = dict(zip(list(zip(pairs.From, pairs.to)),
                  pairs['relation'].tolist()))
    nx.draw_networkx_edge_labels(k_graph, pos=layout, edge_labels=labels,font_size=5,
                                 font_color='g')
    plt.axis('off')
    #plt.show()
    plt.savefig('613.png')
